In [11]:
from bs4 import BeautifulSoup
import pandas as pd
import re

In [12]:
def convert_num_nam(string):
    "Given a string that is a concatenation of a jersey number and the name, return the two substring seperate"

    num = [letter for letter in string if letter.isdigit()]
    num = "".join(map(str, num))
    nam = [letter for letter in string if not letter.isdigit()]
    nam = "".join(map(str, nam))

    return num, nam

In [13]:
with open("./data/quotes-fc-twente-vs-psv-eindhoven.html", 'rb') as file:
    soup = BeautifulSoup(file)

In [34]:
# gets player jersey number and names
num_nams = [player.text for player in soup.find_all(class_=re.compile("LineupPlayerText"))]
numbers = [convert_num_nam(name)[0] for name in num_nams]
names = [convert_num_nam(name)[1] for name in num_nams]

# finds the corresponding ratings
ratings = [rate.text for rate in soup.find_all(class_=re.compile("PlayerRatingStyled"))]
field_ratings = ratings[:22]
sub_ratings = ratings[22:-6]

In [35]:
pd.DataFrame({"name": names, "number": numbers, "rating": field_ratings})

,name,number,rating
0,Unnerstall,1,5.2
1,Smal,5,6.8
2,Pleguezuelo,4,6.8
3,Propper,3,6.6
4,Brenet,20,8.0
5,Sadilek,23,6.9
6,Zerrouki,19,7.0
7,Limnios,18,8.1
8,Vlap,14,7.9
9,Rots,39,8.3
